In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import importlib

from datetime import datetime
from google.cloud.bigquery import Client, LoadJobConfig

sys.path.append('../lib')
import utils
import pandas_utils
import pesys_utils

importlib.reload(utils)
importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

utils.set_secrets_as_env()

In [ ]:
con = pesys_utils.open_pen_connection()

df_ledetid = pandas_utils.pandas_from_sql('../sql/ledetid.sql', con, tuning=200000)
con.close()

In [ ]:
df = df_ledetid.copy()

In [ ]:
df.head()

In [ ]:
df = df.merge(df.groupby(["AR_MANED", "STOENAD", "KRAVTYPE"]).DAGER.quantile(.95),
             on=["AR_MANED", "STOENAD", "KRAVTYPE"],
             suffixes=["", "_KVANTIL95"])

In [ ]:
df.loc[:,"AR"] = pd.to_numeric(df.AR)
df.loc[:,"MANED"] = pd.to_numeric(df.MANED)
df = df[df.AR == 2022]
df = df[df.KRAVTYPE.isin(pesys_utils.fra_bruker)]
df = df[df.STOENAD == "Alderspensjon"]
df = df[df.DAGER < df.DAGER_KVANTIL95] # Fjerne verste outliers - Er det egentlig lurt?

In [ ]:
df.head()

In [ ]:
import plotly.express as px
figs = []
hists = []

for i, kravtype in enumerate(pesys_utils.fra_bruker):
    df_plot = df[df.KRAVTYPE == kravtype]
    if df_plot.shape[0] > 0:
        hist = px.histogram(df_plot, "DAGER", title=kravtype)
        hist.show()
        hists.append(hist)
        
        #fig = px.box(df_plot, "MANED", "DAGER", color="KRAVTYPE", boxmode="group", title=kravtype, height=600)
        #fig.show()
        #figs.append(fig)